In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train.head()

,id,XINGBIE,CSNY,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL,label
0,train_0,1,1038672000,90,90,999,0,99,150,12,1737.0,1,3223.515,801.310,837.000,312.00,312.00,175237,154112.935,2.708,0
1,train_1,2,504892800,90,90,999,0,99,110,0,4894.0,1,18055.195,53213.220,1065.200,795.84,795.84,300237,298252.945,2.979,0
2,train_2,1,736185600,90,90,999,0,99,150,9,10297.0,1,27426.600,13963.140,7230.020,1444.20,1444.20,150237,147339.130,2.708,0
3,train_3,1,428515200,90,90,999,0,99,150,7,10071.5,1,111871.130,99701.265,2271.295,1417.14,1417.14,350237,300653.780,2.708,0
4,train_4,2,544204800,90,90,999,0,99,900,14,2007.0,1,237.000,11028.875,35.780,325.50,325.50,150237,145185.010,2.708,0


In [4]:
test.head()

,id,XINGBIE,CSNY,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DWYJCE,DKFFE,DKYE,DKLL
0,test_0,1,528480000,90,90,999,0,99,159,10,2387.000,1,17146.350,13376.485,1097.000,344.500,344.500,110237,237.000,2.708
1,test_1,1,602265600,90,90,999,0,99,150,11,1237.000,1,3783.385,2145.585,637.000,287.000,287.000,200237,178567.065,2.708
2,test_2,1,31507200,90,90,999,1,99,110,15,5612.904,1,51787.152,35176.592,1115.693,882.108,882.108,150237,146580.203,2.834
3,test_3,2,536428800,90,90,999,0,99,150,10,2977.000,1,785.000,714.885,-2515.105,511.000,511.000,246737,242095.465,2.979
4,test_4,1,86716800,90,90,999,0,99,150,2,4701.500,1,40722.050,27110.840,8163.960,772.740,772.740,100237,72320.445,2.708


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          40000 non-null  object 
 1   XINGBIE     40000 non-null  int64  
 2   CSNY        40000 non-null  int64  
 3   HYZK        40000 non-null  int64  
 4   ZHIYE       40000 non-null  int64  
 5   ZHICHEN     40000 non-null  int64  
 6   ZHIWU       40000 non-null  int64  
 7   XUELI       40000 non-null  int64  
 8   DWJJLX      40000 non-null  int64  
 9   DWSSHY      40000 non-null  int64  
 10  GRJCJS      40000 non-null  float64
 11  GRZHZT      40000 non-null  int64  
 12  GRZHYE      40000 non-null  float64
 13  GRZHSNJZYE  40000 non-null  float64
 14  GRZHDNGJYE  40000 non-null  float64
 15  GRYJCE      40000 non-null  float64
 16  DWYJCE      40000 non-null  float64
 17  DKFFE       40000 non-null  int64  
 18  DKYE        40000 non-null  float64
 19  DKLL        40000 non-nul

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15000 non-null  object 
 1   XINGBIE     15000 non-null  int64  
 2   CSNY        15000 non-null  int64  
 3   HYZK        15000 non-null  int64  
 4   ZHIYE       15000 non-null  int64  
 5   ZHICHEN     15000 non-null  int64  
 6   ZHIWU       15000 non-null  int64  
 7   XUELI       15000 non-null  int64  
 8   DWJJLX      15000 non-null  int64  
 9   DWSSHY      15000 non-null  int64  
 10  GRJCJS      15000 non-null  float64
 11  GRZHZT      15000 non-null  int64  
 12  GRZHYE      15000 non-null  float64
 13  GRZHSNJZYE  15000 non-null  float64
 14  GRZHDNGJYE  15000 non-null  float64
 15  GRYJCE      15000 non-null  float64
 16  DWYJCE      15000 non-null  float64
 17  DKFFE       15000 non-null  int64  
 18  DKYE        15000 non-null  float64
 19  DKLL        15000 non-nul

In [7]:
y = train['label']
sub = test[['id']]
train.drop(['label', 'id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
used_cols = train.columns
test = test[used_cols]

def tpr_weight_funtion(y_predict, train_data):
    y_true = train_data.get_label()
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

X_train, X_valid, y_train, y_valid = train_test_split(train, y, random_state=2020)

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)
params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
#     'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}
valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    num_boost_round=1000000,
    early_stopping_rounds=200,
    verbose_eval=300
)
pred = valid_model.predict(test)
sub['label'] = pred
sub.to_csv('../sub/baseline.csv', index=False)

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[77]	training's auc: 0.976684	valid_1's auc: 0.936583
Evaluated only: auc
